In [1]:
%use kandy

In [2]:
@file:DependsOn("com.ustermetrics:clarabel4j:2.0.0")
@file:DependsOn("com.ustermetrics:clarabel4j-native:2.0.1-0.10.0")
@file:DependsOn("org.ejml:ejml-all:0.44.0")

In [3]:
import com.ustermetrics.clarabel4j.Model
import com.ustermetrics.clarabel4j.Parameters
import com.ustermetrics.clarabel4j.Status.SOLVED
import com.ustermetrics.clarabel4j.Matrix
import com.ustermetrics.clarabel4j.NonnegativeCone
import com.ustermetrics.clarabel4j.SecondOrderCone
import com.ustermetrics.clarabel4j.ZeroCone
import org.ejml.data.DMatrixSparseCSC
import org.ejml.dense.row.factory.DecompositionFactory_DDRM
import org.ejml.ops.DConvertMatrixStruct
import org.ejml.simple.SimpleMatrix

# Portfolio Optimization
        
A long-only investor wishes to maximize the expected portfolio return given a limit on the portfolio risk

$$
\begin{align*}
& \text{maximize} & & \mu^T x \\
& \text{subject to} & & x^T \Sigma x \leq \sigma^2 \\
& & & \mathbf{1} x = 1 \\
& & & x \geq 0
\end{align*}
$$

where $x$ is the unknown vector of portfolio allocations, $\mu$ is the estimated expected return vector, $\Sigma$ is the
estimated covariance matrix, and $\sigma$ is the given limit on the portfolio risk.

In [4]:
// Define portfolio optimization problems
val mu = SimpleMatrix(doubleArrayOf(0.05, 0.09, 0.07, 0.06))
val sigma = SimpleMatrix(
    4, 4, true,
    0.0016, 0.0006, 0.0008, -0.0004,
    0.0006, 0.0225, 0.0015, -0.0015,
    0.0008, 0.0015, 0.0025, -0.001,
    -0.0004, -0.0015, -0.001, 0.01
)
val sigmaLimits = List(20) { 0.0321 + it * (0.06 - 0.0321) / 19 }

In [5]:
// Problem dimension
val n = mu.numRows + 1

In [6]:
// Compute Cholesky decomposition of sigma
val chol = DecompositionFactory_DDRM.chol(n - 1, true)
if (!chol.decompose(sigma.copy().getMatrix()))
    throw IllegalArgumentException("Cholesky decomposition failed")
val upTriMat = SimpleMatrix.wrap(chol.getT(null)).transpose()

// Define second-order cone program
val qMat = mu.negative()
    .concatRows(SimpleMatrix(1, 1))
println("\nqMat")
qMat.print()

val aMatZeroCone = SimpleMatrix.ones(1, n - 1)
    .concatColumns(SimpleMatrix(1, 1))
val aMatNonNeg = SimpleMatrix.identity(n - 1)
    .negative()
    .concatColumns(SimpleMatrix(n - 1, 1))
    .concatRows(SimpleMatrix(1, n - 1).concatColumns(SimpleMatrix.ones(1, 1)))
val aMatSoc = SimpleMatrix(1, n - 1)
    .concatColumns(SimpleMatrix.filled(1, 1, -1.0))
    .concatRows(upTriMat.negative().concatColumns(SimpleMatrix(n - 1, 1)))
val aMat = aMatZeroCone.concatRows(aMatNonNeg).concatRows(aMatSoc)
println("\naMat")
aMat.print()

val bMat = SimpleMatrix(2 * (n - 1) + 3, 1)
bMat[0, 0] = 1.0


qMat
Type = DDRM , rows = 5 , cols = 1
-.05       
-.09       
-.07       
-.06       
 0         

aMat
Type = DDRM , rows = 11 , cols = 5
 1           1           1           1           0         
-1          -0          -0          -0           0         
-0          -1          -0          -0           0         
-0          -0          -1          -0           0         
-0          -0          -0          -1           0         
 0           0           0           0           1         
 0           0           0           0          -1         
-.04        -.015       -.02         .01         0         
-0          -.149248116 -.008040303  .00904534   0         
-0          -0          -.045114893  .016120458  0         
-0          -0          -0          -.097766623  0         


In [7]:
// clarabel4j needs sparse aMat and gMat
val tol = 1e-8
val aSpMat = DConvertMatrixStruct.convert(aMat.ddrm, null as DMatrixSparseCSC?, tol)
println("\naSpMat")
aSpMat.print()


aSpMat
Type = DSCC , rows = 11 , cols = 5 , nz_length = 20
 1           1           1           1               *     
-1               *           *           *           *     
     *      -1               *           *           *     
     *           *      -1               *           *     
     *           *           *      -1               *     
     *           *           *           *       1         
     *           *           *           *      -1         
-.04        -.015       -.02         .01             *     
     *      -.149248116 -.008040303  .00904534       *     
     *           *      -.045114893  .016120458      *     
     *           *           *      -.097766623      *     


In [8]:
// Helper functions
fun convert(matrix: DMatrixSparseCSC): Matrix {
    return Matrix(matrix.getNumRows(), matrix.getNumCols(), getColIdx(matrix), getNzRows(matrix), getNzValues(matrix))
}

fun getNzValues(matrix: DMatrixSparseCSC): DoubleArray {
    if (!matrix.isIndicesSorted()) matrix.sortIndices(null);
    if (matrix.nz_values.size == matrix.nz_length) {
        return matrix.nz_values;
    } else {
        return matrix.nz_values.copyOfRange(0, matrix.nz_length);
    }
}

fun getNzRows(matrix: DMatrixSparseCSC): LongArray {
    if (!matrix.isIndicesSorted()) matrix.sortIndices(null);
    if (matrix.nz_rows.size == matrix.nz_length) {
        return toLongArray(matrix.nz_rows);
    } else {
        return toLongArray(matrix.nz_rows.copyOfRange(0, matrix.nz_length));
    }
}

fun getColIdx(matrix: DMatrixSparseCSC): LongArray {
    return toLongArray(matrix.col_idx);
}

fun toLongArray(arr: IntArray): LongArray {
    return arr.map { it.toLong() }.toLongArray()
}

In [9]:
val muPortfolio = mutableListOf<Double>()
val sigmaPortfolio = mutableListOf<Double>()
lateinit var xMat: SimpleMatrix

// Compute efficient frontier
sigmaLimits.forEach { sigmaLimit ->

    // Set sigma limit
    bMat[n, 0] = sigmaLimit

    Model().use { model ->

        // Create and set parameters
        val parameters = Parameters.builder()
            .verbose(true)
            .build()
        model.setParameters(parameters)

        // Set up model
        model.setup(
            qMat.getDDRM().data, convert(aSpMat), bMat.ddrm.data,
            listOf(ZeroCone(1), NonnegativeCone(n.toLong()), SecondOrderCone(n.toLong()))
        )

        // Optimize model
        val status = model.optimize()
        if (status != SOLVED)
            throw IllegalStateException("Optimization failed")

        // Get solution
        xMat = SimpleMatrix(model.x())
            .extractMatrix(0, n - 1, 0, 1)

        // Compute portfolio return and risk
        muPortfolio.add(mu.transpose().mult(xMat)[0, 0])
        sigmaPortfolio.add(sqrt(xMat.transpose().mult(sigma).mult(xMat)[0, 0]))
    }
}

In [10]:
// Plot efficient frontier
plot {
    layout.title = "Efficient frontier"
    smoothLine(sigmaPortfolio, muPortfolio, method = SmoothMethod.LOESS(span = 0.1)) {
        color = Color.GREEN
    }
    points {
        x(sigmaPortfolio) {
            axis {
                name = "Risk"
            }
        }
        y(muPortfolio) {
            axis {
                name = "Return"
            }
        }
        size = 3.0
        color = Color.GREEN
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="VY2nFK"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Efficient frontier"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Risk",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"Return",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[0.032100000284819914,0.0323818184565676,0.03266363662831529,0.03294545480006298,0.03322727297181067,0.03350909114355836,0.033790909315306046,0.034072727487053735,0.03435454565880142,0.03463636383054911,0.0349181820022968,0.03520000017404449,0.03548181834579218,0.03576363651753987,0.03604545468928756,0.03632727286103525,0.03660909103278294,0.03689090920453063,0.03717272737627832,0.037454545548026005,0.037736363719773694,0.03801818189152138,0.03830000006326907,0.03858181823501676,0.03886363640676445,0.03914545457851214,0.039427272750259826,0.039709090922007514,0.0399909090937552,0.04027272726550289,0.04055454543725058,0.04083636360899827,0.04111818178074596,0.041399999952493646,0.041681818124241335,0.041963636295989024,0.04224545446773671,0.0425272726394844,0.04280909081123209,0.04309090898297978,0.04337272715472747,0.043654545326475155,0.043936363498222844,0.04421818166997053,0.04449999984171822,0.04478181801346591,0.0450636361852136,0.04534545435696129,0.045627272528708976,0.045909090700456665,0.04619090887220435,0.04647272704395204,0.04675454521569973,0.047036363387447426,0.047318181559195115,0.0475999997309428,0.04788181790269049,0.04816363607443818,0.04844545424618587,0.04872727241793356,0.049009090589681246,0.049290908761428935,0.049572726933176624,0.04985454510492431,0.050136363276672,0.05041818144841969,0.05069999962016738,0.05098181779191507,0.051263635963662756,0.051545454135410444,0.05182727230715813,0.05210909047890582,0.05239090865065351,0.052672726822401206,0.052954544994148894,0.05323636316589658,0.05351818133764427,0.05379999950939196,0.05408181768113965,0.05436363585288734,0.054645454024635026,0.054927272196382715,0.055209090368130403,0.05549090853987809,0.05577272671162578,0.05605454488337347,0.05633636305512116,0.05661818122686885,0.056899999398616535,0.057181817570364224,0.05746363574211191,0.0577454539138596,0.05802727208560729,0.05830909025735498,0.05859090842910267,0.058872726600850356,0.059154544772598044,0.05943636294434573,0.05971818111609342,0.05999999928784111],
"y":[0.05853416876443665,0.059440487929273755,0.06032194632504581,0.061153683182687775,0.061910837733134605,0.06256854920732124,0.06310625858364831,0.0635362965789284,0.06388617572003225,0.06418349011006873,0.06445583385214675,0.06472973512720945,0.06501436308477247,0.06530476406452082,0.06559557437757309,0.06588143033504779,0.06615703069056057,0.06642002410003624,0.06667222428775085,0.06691575955601627,0.06715275820714432,0.06738534715408538,0.06761511093900677,0.06784227091922221,0.06806683767799543,0.06828882179859018,0.06850823386427017,0.06872500188291256,0.06893866696705846,0.06914865701363399,0.06935439990540616,0.06955532352514195,0.06975080826681035,0.06993980253428135,0.07012102413621715,0.07029318871787,0.07045501192449216,0.07060528581418715,0.0707443604293319,0.07087404041125799,0.07099618610652061,0.07111265786167506,0.0712253062568923,0.07133531446380605,0.07144278141829782,0.07154770672115587,0.07165008997316846,0.07174993081743726,0.07184727771182313,0.07194232194618401,0.07203528079671816,0.0721263715396238,0.07221581145109915,0.0723037988746580

In [11]:
// Plot asset allocation with 6% risk
val assets = MutableList(n - 1) { i -> "Asset ${i + 1}" }
val allocations = MutableList(n - 1) { i -> 100 * xMat[i, 0] }

plot {
    layout.title = "Asset allocation with 6% risk"
    bars {
        x(assets) { axis.name = "" }
        y(allocations) { axis.name = "Allocation (%)" }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="iIkCeQ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Asset allocation with 6% risk"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":""
},{
"aesthetic":"y",
"name":"Allocation (%)",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["Asset 1","Asset 2","Asset 3","Asset 4"],
"y":[1.173937830603296E-6,29.676395185651316,66.34060318111547,3.9830004592953867]
},
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"x"
},{
"type":"float",
"column":"y"
}]
}
}],
"spec_id":"5"
};
 var containerDiv = document.getElementById("iIkCeQ");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Asset 1 
 
 
 
 
 
 
 
 
 Asset 2 
 
 
 
 
 
 
 
 
 Asset 3 
 
 
 
 
 
 
 
 
 Asset 4 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 
 
 Asset allocation with 6% risk 
 
 
 
 
 Allocation (%)